In [ ]:
import torch
from torch import Tensor
from torch.distributions import Exponential
import numpy as np
import torch
from sklearn.datasets import make_circles
import pandas as pd
import matplotlib.pyplot as plt
import random

In [ ]:
values = []
labels = []

n=1
while n < 1000: 
    x = random.randint(0,1)
    if x == 0:
        values.append(numpy.random.poisson(2))
    if x == 1:
        values.append(numpy.random.poisson(6))

    labels.append(x)

    n = n + 1

values = numpy.array(values)
labels = numpy.array(labels)
print(labels)

data = pd.DataFrame({"values": values, "labels": labels})
X = torch.from_numpy(values).type(torch.float)
y = torch.from_numpy(labels).type(torch.float)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, # 20% test, 80% train
                                                    random_state=42) # make the random split reproducible

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
def sample_poisson_relaxed(lmbd, num_samples=1000, temperature = 1e-2):
    sampler = torch.empty(num_samples)
    z = Exponential(lmbd).rsample(sampler.size())
    t = torch.cumsum(z, 0)
    relaxed_indicator = torch.sigmoid((1.0 - t) / temperature)
    N = relaxed_indicator.sum()
    return N

from torch.distributions.multivariate_normal import MultivariateNormal
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
class CircleModelV0(nn.Module):
    def __init__(self):
        super().__init__()
        # 2. Create 2 nn.Linear layers capable of handling X and y input and output shapes
        self.layer_1 = nn.Linear(in_features=1, out_features=20) # takes in 2 features (X), produces 5 features
        self.layer_2 = nn.Linear(in_features=20, out_features=10) # takes in 5 features, produces 1 feature (y)
        self.layer_3 = nn.Linear(in_features=10, out_features=1) # takes in 5 features, produces 1 feature (y)
        # self.layer_4 = nn.Linear(in_features=10, out_features=1) # takes in 5 features, produces 1 feature (y)
        

    
    # 3. Define a forward method containing the forward pass computation
    def forward(self, x):
        # print(x)
        # x.numpy
        # print(x)
        # print(x.size())
        for i in range(x.size(dim=0)):
            y = x[i].item()
            # print(y) 
            x[i] = sample_poisson_relaxed(y+0.0001) #added a small decimal to counteract errors arising from a lambda=0
        
        # print(x)
        rx = x
        rx = rx.view(rx.size(dim =0),1)
        # print(rx)
        # print(rx.size())
        

        # Return the output of layer_2, a single feature, the same shape as y
        # rx = self.layer_1(rx)
        # print(1)
        # rx = self.layer_2(torch.relu(rx))
        # print(2)
        # rx = self.layer_3(torch.relu(rx))
        # return(rx)
        return self.layer_3(torch.relu(self.layer_2(torch.relu(self.layer_1(rx))))) # computation goes through layer_1 first then the output of layer_1 goes through layer_2

def makeModel():
    return CircleModelV0().to(device)
# 4. Create an instance of the model and send it to target device
# model_0 = CircleModelV0().to(device)
# model_0

In [ ]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

In [ ]:
epochs = 1000
torch.manual_seed(42)

# Put data to target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

noises = []
accuracy = []
finalaccuracy = []
finalnoise = []

# Build training and evaluation loop
for x in range(10):
    model_0 = makeModel()
    print(model_0)
    loss_fn = nn.BCEWithLogitsLoss() # BCEWithLogitsLoss = sigmoid built-in

    # Create an optimizer
    optimizer = torch.optim.SGD(params=model_0.parameters(), 
                                lr=0.1)
    for epoch in range(epochs):
        ### Training
        model_0.train()

        # 1. Forward pass (model outputs raw logits)
        y_logits = model_0(X_train).squeeze() # squeeze to remove extra `1` dimensions, this won't work unless model and data are on same device 
        y_pred = torch.round(torch.sigmoid(y_logits)) # turn logits -> pred probs -> pred labls
    
        # 2. Calculate loss/accuracy
        # loss = loss_fn(torch.sigmoid(y_logits), # Using nn.BCELoss you need torch.sigmoid()
        #                y_train) 
        loss = loss_fn(y_logits, # Using nn.BCEWithLogitsLoss works with raw logits
                    y_train) 
        acc = accuracy_fn(y_true=y_train, 
                        y_pred=y_pred) 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backwards
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        ### Testing
        model_0.eval()
        with torch.inference_mode():
            # 1. Forward pass
            test_logits = model_0(X_test).squeeze() 
            test_pred = torch.round(torch.sigmoid(test_logits))
            # 2. Caculate loss/accuracy
            test_loss = loss_fn(test_logits,
                                y_test)
            test_acc = accuracy_fn(y_true=y_test,
                                y_pred=test_pred)

        # Print out what's happening every 10 epochs
        if epoch % 10 == 0:
            print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")
        noises.append(-x)
        accuracy.append(acc)
    finalaccuracy.append(acc)
    finalnoise.append(-x)